In [14]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms,datasets
from torch import optim
from visdom import Visdom

In [10]:
mnist_train = datasets.MNIST('./datasets/mnist_data',True,transform = transforms.Compose([
    transforms.ToTensor()
]),download = True)
mnist_train = DataLoader(mnist_train,batch_size = 32,shuffle = True)

mnist_test = datasets.MNIST('./datasets/mnist_data/',False,transform = transforms.Compose([
    transforms.ToTensor()
]),download = True)
mnist_train = DataLoader(mnist_train,batch_size = 32,shuffle = True)

In [12]:
# AE
class AE(nn.Module):

    def __init__(self):
        super(AE, self).__init__()

        # [b, 784] => [b, 20]
        self.encoder = nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 20),
            nn.ReLU()
        )
        # [b, 20] => [b, 784]
        self.decoder = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 784),
            nn.Sigmoid()
        )

    def forward(self, x):
        """
        :param x: [b, 1, 28, 28]
        :return:
        """
        batch_size = x.size(0)
        # flatten
        x = x.view(batch_size, 784)
        # encoder
        x = self.encoder(x)
        # decoder
        x = self.decoder(x)
        # reshape
        x = x.view(batch_size, 1, 28, 28)

        return x, None

In [ ]:
# train
device = torch.device('cuda')
model = AE().to(device)
criteon = nn.MSEloss()
optimizer = optim.Adam(model.parameters(),lr = 1e-3)

print(model)

for epoch in range(1000):
    for batch_idx,(x,_) in enumerate(mnist_train):
        x = x.to(device)
        x_hat = model(x)
        loss = criteon(x_hat,x)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()